---

# Lib

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import re

from preprocessing.clean import vn_text_clean
from preprocessing.tokenize import vn_word_tokenize
from preprocessing.remove_stopwords import remove_stopwords

---

# Read file

In [ ]:
df = pd.read_csv("../data/processed/train_processed.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [ ]:
df.head()

---

# Model

## Vectorize

## Svm

## Logistic regression

## Xgboost